Experimento GA.04 - caixeiro astronauta
========================================



## Introdução



Os experimentos do caixeiro viajante que até então vimos tratavam-se de deslocamentos descritos em 2 coordenadas. No entanto, **no mais novo episódio dessa série de experimentos**, nosso querido caixeiro fará uma viagem a algumas **estações espaciais** &#128104;&#8205;&#128640; &#128640; o que implica dizer que a sua localização será agora dada por um sistema de 3 coordenadas(&#128558;).

Nesse notebook, trataremos, então, de um número $n\geq 7$ de coordenadas $(x,y,z)$ de estações espaciais a serem visitadas.

## Objetivo



Encontrar o caminho de menor distância no problema do caixeiro viajante astronauta.



## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random
from itertools import permutations

from funcoes import cria_estacoes_espaciais as cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv_astronauta
from funcoes import funcao_objetivo_cv_astronauta
from funcoes import selecao_torneio_min as funcao_selecao # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



Para iniciarmos a discussão é valido relembrar que o caixeiro só anda em linha reta e apenas entre duas estações. O caixeiro começa e termina seu trajeto na mesma estação e, fora a estação inicial, ele não visita nenhuma outra estação mais de uma vez.

Diferentemente do experimento GA.03, nesse experimento em que estamos voltamos a buscar pela menor distância a ser percorrido por nosso amiguinho, sendo, assim, um poblema de minimização.

A principal diferença entre o algoritmo original do caixeiro viajante e o do algoritmo do caixeiro viajante astronauta(que agora construimos), trata-se da mudança nas funções que geram as cidades, que medem a distância e consequentemente nas funções objetivo.

Dessa forma, tais alterações foram feitas em sua maioria no arquivo `funcoes.py` no qual podemos ver a criações de funções como a `cria_estacoes_espaciais`,`distancia_entre_dois_pontos_em_tres_coordenadas`,`funcao_objetivo_pop_cv_astronauta`e `funcao_objetivo_cv_astronauta`; que representaram, por sua vez, novidades na seção de importação desse notebook.

Por questão de simplificação continuaremos chamandos as "estações espaciais" a serem visitadas pelo caixeiro de "cidades" o que justifica, por exemplo, a importação da função `cria_estacoes_espaciais` como `cria_cidades`, o que nos permitirá utilizar os códigos feitos anteriormente fazendo apenas pequenas alterações

In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 1000
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES = 10
CIDADES = cria_cidades(NUMERO_DE_CIDADES)
CIDADES

{'Estação 0': (0.09652190735451827, 0.7781701459865531, 0.49025939453774636),
 'Estação 1': (0.07560417405281661, 0.7304677540615218, 0.16795897119619307),
 'Estação 2': (0.5822215730574914, 0.005839578033372428, 0.2279921249121134),
 'Estação 3': (0.016571694022325834, 0.6692393052806365, 0.40336242658512567),
 'Estação 4': (0.07412686526996815, 0.7474799883896465, 0.007193969263261546),
 'Estação 5': (0.4990360067676244, 0.5370993414208183, 0.9171161959825356),
 'Estação 6': (0.024517433927354726, 0.6320297055832288, 0.197889586472759),
 'Estação 7': (0.7240434476178683, 0.8831323993213072, 0.10611464462576747),
 'Estação 8': (0.30991842015960913, 0.23386491685164879, 0.3470115258357924),
 'Estação 9': (0.48506879999909713, 0.8600396246716057, 0.1427915523668598)}

Com o código acima podemos ver que foi realizada com sucesso a troca da representação das cidades de 2 coordenadas para 3 coordenadas.

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv_astronauta(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv_astronauta(individuo, CIDADES)

Definidas também as nossas funções locais, podemos, então, seguir para a busca por algoritmos genéticos&#129321;.

**Busca por algoritmos genéticos:**

In [4]:
populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf") #infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:
        posicao  = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness

Como o problema do caixeiro viajante pode ser considerado um problema NP difícil, só conseguimos, de fato, testar se uma determinada rota é a melhor possível se todas as rotas forem testadas.

Dessa forma, achamos interessante propor também uma solução por busca exaustiva (dado que nesse caso ainda é possível fazê-la sem exigir tanto dos nossos computadores) e comparar o resultado obtido dessa forma com o obtido por meio de algoritmos genéticos. 

**Busca testando todas as permutações:**

In [5]:
melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis 
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

**Comparando resultados:**

In [6]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos")
print(melhor_individuo_ja_visto, "com distancia:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos
['Estação 6', 'Estação 3', 'Estação 0', 'Estação 5', 'Estação 8', 'Estação 2', 'Estação 7', 'Estação 9', 'Estação 4', 'Estação 1'] com distancia: 3.9149076627735884

Melhor individuo obtido por busca exaustiva:
('Estação 0', 'Estação 3', 'Estação 6', 'Estação 1', 'Estação 4', 'Estação 9', 'Estação 7', 'Estação 2', 'Estação 8', 'Estação 5') com distância: 3.9149076627735884


## Conclusão



Dado que o nosso principal objetivo nesse desafio era encontrar, por meio de algoritmos genéticos, o caminho com menor distância necessária para o caixeiro viajante astronauta percorrer todas as estações espaciais(descritas em coordenadas x, y e z) podemos considerá-lo atingido com sucesso! Isso se dá porque, quando comparado com o resultado obtido de busca exaustiva, podemos ver que o algoritmo genético obteve resultados tão bons quanto!

Durante esse notebook pudemos entender como modificar e acrescentar pequenos detalhes no nosso código de modo que ele passasse a servir para outro problema diferente alterando ou acrescentando uma pequena quantidade de elementos!

Com isso o nosso conhecimento sobre algoritmos genéticos, suas aplicações, bem como a capacidade de adaptarmos e manipularmos códigos ficam cada vez mais robustos e estou animada para exercer essas habilidades que estão sendo construídos no trabalho final da disciplina!&#128516;

## Playground



Todo código de teste que não faz parte do seu experimento deve vir aqui. Este código não será considerado na avaliação.

